In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["pdf.fonttype"] = 42
import basty.project.experiment_processing as experiment_processing
import seaborn as sns

sns.set_theme(style="dark")
from basty.utils import misc
from process_results import BehaviorData
from filter_flies import FilterData
from input import Input

import sys

sys.path.append(r"C:\Users\Grover\Documents\GitHub>basty")

In [2]:
CONFIG_PATH = r"Z:\mfk\basty-projects\main_cfg.yaml"
project = experiment_processing.Project(CONFIG_PATH)
FPS = project.fps
FIG_PATH = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs"

RESULTS_FOLDER = r"Z:\mfk\basty-projects\backup-allnohaltere\results\semisupervised_pair_kNN\predictions.15NN.neighbor_weights-distance.neighbor_weights_norm-log_count.activation-standard.voting-soft.voting_weights-None\exports"

BODY_PART_SETS = {"ProboscisPumping": "prob"}

TMP_RESULT_FOLDER = r"Z:\mfk\basty-projects"

BEHAVIORS = [
    "Idle&Other",
    "PosturalAdjustment&Moving",
    "Feeding",
    "Grooming",
    "ProboscisPumping",
]

In [3]:
io_process = Input(project, RESULTS_FOLDER, TMP_RESULT_FOLDER)
expt_info_df = io_process.load_expt_info()
df_data = io_process.load_predictions()
df_data.drop(["HaltereSwitch", "Noise"], axis=1, inplace=True)
io_process.create_binary_masks_subfolders(BEHAVIORS)

In [4]:
llh = misc.get_likelihood(project.data_path_dict, CONFIG_PATH, n_worked)

Loading existing llh_df.pickle file...


In [5]:
process_data = BehaviorData(df_data, binary_mask_threshold=0.8)

In [6]:
process_data.process_expt_names_parallel(
    llh, io_process.get_binary_mask_subfolder("ProboscisPumping")
)

File Fly05182022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.File Fly06142022_SD_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly05202022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly05222022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly05252022_SD_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly05272022_SD_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly06072022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly06082022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.

File Fly06152022_6d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly06202022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly06212022_5d_median6_threshold0.8.pkl already exists. Skipping calculation.
File Fly06182022_4d_median6_threshold0.8.pkl already exists. Skipping calculat

In [7]:
llh = llh[llh["ExptNames"].isin(df_data.ExptNames.unique())]

In [8]:
newDf = pd.merge(df_data, expt_info_df, left_on="ExptNames", right_on="ExptNames")

In [9]:
# Create a folder named binary mask and a subfolder for each predicted behavior.
# Within that folder save binary mask generated using the likelihood score

In [10]:
wt_all = newDf[newDf["SD"] == False]
male_wt = wt_all[wt_all["Sex"] == "M"]
female_wt = wt_all[wt_all["Sex"] == "F"]

In [14]:
data = BehaviorData(behaviors=BEHAVIORS)

,start_index,stop_index,region,bf98aa1786,6b6b013840,736e0c8583,80c4b09f71,e38dd6300f,9e57f9fab8,b11bab0d11,4edfe1945a,f06a9e0a70,42b3adc018,2bbee6c03d,bfe5b3e481,3a8fb82d45
0,2921,3268,Fly05182022_5d,"[758.6258810125291, 758.6896228554348, 758.773...","[638.0838725566864, 637.9911467234293, 637.647...","[592.2871385216713, 592.0673806567987, 591.939...","[587.5258777538935, 587.4953110615412, 587.329...","[600.2830431461334, 600.0328272978464, 599.782...","[549.4471956032017, 549.7231232660512, 549.810...","[834.2605776906113, 834.0830436815627, 833.875...","[991.2942326872347, 991.2833334249085, 991.126...","[813.7763529649935, 813.7781676159844, 813.652...","[80.44709750226653, 80.42179532363298, 80.3016...","[38.90913164151707, 38.60293396944258, 38.3294...","[145.5113712455229, 145.17950994787546, 145.02...","[915.7810046516133, 915.6801057550965, 915.637..."
1,3015,3337,Fly05182022_5d,"[758.2635271946589, 758.6790062983831, 759.343...","[636.865932226181, 636.4138186573982, 636.0373...","[593.3076470692953, 593.0660186211268, 592.824...","[587.2004273037115, 587.2150823126236, 587.452...","[600.8599585294724, 600.7712555428346, 600.682...","[550.5488957961401, 550.5602042873701, 550.771...","[834.756435073468, 834.5950247506487, 834.5905...","[990.2332009703613, 990.260765241153, 990.5284...","[814.9458733108527, 814.8881150387854, 814.965...","[78.73572879168856, 78.52161695230708, 78.5408...","[37.421546871315, 37.455984289107455, 37.51377...","[144.91461752756064, 145.00640634170725, 144.9...","[916.8529863615873, 916.8515164922247, 916.901..."
2,3049,3354,Fly05182022_5d,"[759.3077299942573, 757.4862793236971, 755.999...","[648.1716264088949, 646.2815745075544, 644.155...","[591.0555632462105, 590.8307492683331, 590.619...","[587.6654513676962, 587.3899451295534, 587.471...","[599.7426815629005, 599.6975306272507, 599.539...","[547.6847516496977, 547.3742707421383, 547.173...","[833.4850697975897, 833.1313953514654, 833.038...","[998.3359585382989, 995.7235243337489, 993.213...","[812.1883225446237, 811.9456388890268, 811.693...","[89.87807457742481, 87.83315904313895, 85.4491...","[40.91359645141134, 40.98626609422336, 41.2731...","[146.02003278362406, 146.11778886238469, 146.2...","[914.9604916392724, 914.903447254525, 914.9340..."
3,3060,3394,Fly05182022_5d,"[755.6794727966189, 755.4996426254511, 755.274...","[640.9506965478262, 641.2885080178579, 641.608...","[590.7927434444427, 590.7230416735014, 590.627...","[587.3997285366058, 587.3868263562521, 587.157...","[599.5620582501093, 599.3672413627306, 599.129...","[547.0130272946941, 546.8283525121709, 546.609...","[833.1113411732489, 833.052816897502, 832.8232...","[990.8931632680284, 990.9746013510915, 991.009...","[811.6020661156631, 811.3336780449489, 811.010...","[83.57850051151549, 84.06557854057206, 84.1606...","[41.3277935458006, 41.46941027827967, 41.42932...","[145.88928557810198, 146.00388473621297, 146.1...","[913.8551437732976, 913.7154755493824, 913.535..."
4,3104,3468,Fly05182022_5d,"[755.4611857831478, 755.8999953369299, 756.153...","[636.1042799552282, 636.549527088801, 636.3900...","[591.4155917068323, 591.635134935379, 591.6537...","[587.4608536909024, 587.8117415805658, 587.998...","[598.88236442705, 598.8248790999254, 598.58449...","[549.2257466316223, 549.2965656121572, 549.302...","[833.5962192411781, 833.9992664445108, 834.143...","[987.5982271157837, 988.2206754502492, 988.311...","[812.5939989839926, 812.5995033362691, 812.426...","[77.93312728272355, 78.09147833400941, 77.6735...","[38.95736267626434, 39.18049516059115, 39.3118...","[144.13879219048002, 144.2031883319876, 144.17...","[914.6203983077863, 914.7134187691836, 914.802..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,959717,960142,Fly05182022_5d,"[647.7980016767979, 648.0658544053634, 648.468...","[178.37109940250716, 178.47062985102335, 178.3...","[803.6159871816635, 803.8594224452972, 803.894...","[256.122812350591, 255.98054448763529, 255.699...","[

In [ ]:
newDf

In [ ]:
newDf

In [ ]:
filterer = FilterData(newDf, llh)

In [ ]:
filterer.plot_mean_likelihood()

In [ ]:
binary_mask = filterer.create_binary_masks(llh, BODY_PART_SETS, 0.7)

In [ ]:
masked_data_df = filterer.apply_binary_masks(binary_mask, newDf)

In [ ]:
masked_data_df

In [ ]:
binary_mask

In [ ]:
masked_data_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is called 'masked_data_df'
# and has columns 'ExptNames', 'ValueColumn' (replace with your actual column name)
# Add row numbers if they don't exist
masked_data_df["Row"] = masked_data_df.index

# Pivot the DataFrame
pivoted_df = masked_data_df.pivot(
    index="ExptNames", columns="Row", values="ProboscisPumping"
)

# Create a heatmap using Seaborn, handling NaN values with a different color (e.g., gray)
plt.figure(figsize=(16, 9))
sns.heatmap(
    pivoted_df,
    cmap="coolwarm",
    linewidths=0.5,
    cbar_kws={"ticks": [0, 1]},
    mask=pivoted_df.isna(),
    linecolor="gray",
)

# Set plot attributes
plt.xlabel("Rows")
plt.ylabel("ExptNames")
plt.title("Heatmap of Data")

# Save the plot as a PNG file
FIG_PATH = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs"
plt.savefig(f"{FIG_PATH}\\heatmap.png", dpi=300, bbox_inches="tight")

# Display the plot in the notebook (optional)
plt.show()

In [ ]:
plot_folder = r"C:\Users\Grover\Documents\GitHub\deepsleepfigs\score_plots"
if not os.path.isdir(plot_folder):
    os.makedirs(plot_folder)

In [ ]:
male_wt["Time"] = male_wt["Idx"].apply(BehaviorData.get_time_stamp)
male_wt.index = pd.to_datetime(list(male_wt["Time"]))
rate = "1S"
male_wt_rs = BehaviorData.resample_df(male_wt, rate, BEHAVIORS)
male_wt_rs["TimeStamp"] = male_wt_rs.index

In [ ]:
male_wt

In [ ]:
llh["Time"] = llh["Idx"].apply(BehaviorData.get_time_stamp)

In [ ]:
BehaviorData.pivot_and_plot(male_wt_rs, "mal_b_scr_", plot_folder, rate, BEHAVIORS)

In [ ]:
import os

In [ ]:
os.listdir(plot_folder)

In [ ]:
df_pivoted = male_wt_rs.pivot(
    index="ExptNames", columns="TimeStamp", values="ProboscisPumping"
)
a4_dims = (25.7, 5.27)
fig, ax = plt.subplots(figsize=a4_dims)
plt.title("ProboscisPumping")
ax = sns.heatmap(df_pivoted, cmap="YlGnBu")

In [ ]:
male_wt_rs

In [ ]:
result_df_parallel = BehaviorData.process_expt_names_parallel(male_wt, llh)
print(result_df_parallel)

In [ ]:
unique_expt_names = male_wt["ExptNames"].unique()

unmasked_data = np.vstack(
    [result_df_parallel[f"{expt_name}_unmasked"] for expt_name in unique_expt_names]
)
masked_data = np.vstack(
    [result_df_parallel[f"{expt_name}_masked"] for expt_name in unique_expt_names]
)

# Plot the heatmap
plt.figure(figsize=(10, 3))
sns.heatmap(data, cmap="YlGnBu")
plt.xlabel("Data")
plt.ylabel("Arrays")
plt.yticks([0.5, 1.5], ["Array 1", "Array 2"], rotation=0)
plt.show()

In [ ]:
unique_expt_names = male_wt["ExptNames"].unique()

a4_dims = (25.7, 5.27)
# Plot the unmasked data heatmap
# plt.figure(figsize=(a4_dims))
# sns.heatmap(unmasked_data, cmap='YlGnBu', yticklabels=unique_expt_names)
# plt.xlabel('Data')
# plt.ylabel('ExptNames')
# plt.title('Unmasked Data')
# plt.savefig(os.path.join(plot_folder,'unmasked_data_heatmap_full_07.png'), dpi=300)  # Save the figure as a PNG file
# plt.show()

In [ ]:
# Plot the masked data heatmap
plt.figure(figsize=(a4_dims))
sns.heatmap(masked_data, cmap="YlGnBu", yticklabels=unique_expt_names)
plt.xlabel("Data")
plt.ylabel("ExptNames")
plt.title("Masked Data")
plt.savefig(os.path.join(plot_folder, "masked_data_heatmap_full_09.png"), dpi=300)
plt.show()

In [ ]:
snap_paths = [
    {expt_names: os.path.join(project.project_path, expt_names, "snap_stft.pkl")}
    for expt_names in project.expt_names
]
snap_col_names = [
    os.path.join(project.project_path, expt_names, "ftname_to_snapft.yaml")
    for expt_names in project.expt_names
]

In [ ]:
df_data

In [ ]:
df_data